In [1]:
def getRotationMatrixFromVector(rotation_vector, return_size):
    q1 = rotation_vector[:, 0]
    q2 = rotation_vector[:, 1]
    q3 = rotation_vector[:, 2]
    q0 = rotation_vector[:, 3]
    sq_q1 = 2 * q1 * q1
    sq_q2 = 2 * q2 * q2
    sq_q3 = 2 * q3 * q3
    q1_q2 = 2 * q1 * q2
    q3_q0 = 2 * q3 * q0
    q1_q3 = 2 * q1 * q3
    q2_q0 = 2 * q2 * q0
    q2_q3 = 2 * q2 * q3
    q1_q0 = 2 * q1 * q0
    if return_size == 16:
        rotation_matrix = np.zeros([np.shape(rotation_vector)[0], 16])
        rotation_matrix[:,0] = 1 - sq_q2 - sq_q3
        rotation_matrix[:,1] = q1_q2 - q3_q0
        rotation_matrix[:,2] = q1_q3 + q2_q0
        rotation_matrix[:,4] = q1_q2 + q3_q0
        rotation_matrix[:,5] = 1 - sq_q1 - sq_q3
        rotation_matrix[:,6] = q2_q3 - q1_q0
        rotation_matrix[:,8] = q1_q3 - q2_q0
        rotation_matrix[:,9] = q2_q3 + q1_q0
        rotation_matrix[:,10] = 1 - sq_q1 - sq_q2
        rotation_matrix[:, 15] = 1
        rotation_matrix = rotation_matrix.reshape([np.shape(rotation_vector)[0], 4, 4])
    elif return_size == 9:
        rotation_matrix = np.zeros([np.shape(rotation_vector)[0], 9])
        rotation_matrix[:, 0] = 1 - sq_q2 - sq_q3
        rotation_matrix[:, 1] = q1_q2 - q3_q0
        rotation_matrix[:, 2] = q1_q3 + q2_q0
        rotation_matrix[:, 3] = q1_q2 + q3_q0
        rotation_matrix[:, 4] = 1 - sq_q1 - sq_q3
        rotation_matrix[:, 5] = q2_q3 - q1_q0
        rotation_matrix[:, 6] = q1_q3 - q2_q0
        rotation_matrix[:, 7] = q2_q3 + q1_q0
        rotation_matrix[:, 8] = 1 - sq_q1 - sq_q2
        rotation_matrix = rotation_matrix.reshape([np.shape(rotation_vector)[0], 3, 3])
    return rotation_matrix
def getOrientation(rotation_matrix):
    orientation = np.zeros(3)  # Azimuth, pitch, roll
    orientation[0] = np.arctan2(rotation_matrix[0][1], rotation_matrix[1][1])
    orientation[1] = np.arcsin(-rotation_matrix[2][1])
    orientation[2] = np.arctan2(-rotation_matrix[2][0], rotation_matrix[2][2])
    return orientation

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib widget

In [3]:
data_path = "data/mag/210311/"
file_list = os.listdir(data_path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]
print(file_list_csv)

['1233_00.csv', '1233_01.csv', '1250_00.csv', '1250_01.csv', '1251_00.csv', '1251_01.csv']


In [4]:
raw_list = []
for file in file_list_csv:
    raw_list.append(pd.read_csv(data_path + file, header=None).to_numpy())

In [5]:
time_list = []
mag_raw_list = []
game_vec_list = []
for raw in raw_list:
    time_col = [0]
    mag_raw_col = [7, 8, 9, 10, 11, 12]
    game_vec_col = [13, 14, 15, 16] 
    time_list.append(raw[:,time_col].reshape((-1,1)).astype(np.float32))
    mag_raw_list.append(raw[:,mag_raw_col].astype(np.float32))
    game_vec_list.append(raw[:,game_vec_col].astype(np.float32))

In [16]:
mag_list = []
for mag_raw in mag_raw_list:
    mag_list.append(mag_raw[:, :3] - mag_raw[:, 3:])
tilted_mag_list = []
for mag, game_vec in zip(mag_list, game_vec_list):
    tilted_mag = np.matmul(getRotationMatrixFromVector(game_vec, 9), mag.reshape([-1, 3, 1]))
    tilted_mag_list.append(tilted_mag.reshape([-1, 3]))
feat_list = []
for time, tilted_mag in zip(time_list, tilted_mag_list):
    mag_xy = np.sqrt(np.sum(tilted_mag[:,:2]**2, axis=1)).reshape([-1, 1])
    feat_list.append(np.concatenate((time, mag_xy, tilted_mag[:, 2].reshape([-1, 1])), axis = 1))

In [32]:
plt.figure()
color = ['indigo', 'm']
for idx, feat in enumerate(feat_list):
    plt.scatter(feat[:,1], feat[:,2], label=file_list_csv[idx][:-4], c = color[idx])
    if idx==1:
        break
plt.title('mag comparison')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plt.figure()
color = ['indigo', 'm']
for idx, feat in enumerate(feat_list):
    if idx<=1:
        continue
    plt.scatter(feat[:,1], feat[:,2], label=file_list_csv[idx][:-4], c = color[idx-2])
    if idx==3:
        break
plt.title('mag comparison')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plt.figure()
color = ['indigo', 'm']
for idx, feat in enumerate(feat_list):
    if idx<=3:
        continue
    plt.scatter(feat[:,1], feat[:,2], label=file_list_csv[idx][:-4], c = color[idx-4])
    if idx==5:
        break
plt.title('mag comparison')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plt.figure()
color = ['indigo', 'indigo', 'm', 'm', 'gold','gold']
for idx, feat in enumerate(feat_list):
    plt.scatter(feat[:,1], feat[:,2], label=file_list_csv[idx][:-4], c = color[idx])
plt.title('mag comparison')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
plt.figure()
color = ['indigo', 'indigo', 'm', 'm', 'gold','gold']
for idx, feat in enumerate(feat_list):
    addi = np.shape(feat)[0]%50
    new_feat = feat[:-addi].reshape([-1,50,3])
    new_feat_mean = np.mean(new_feat, axis=1)
    plt.scatter(new_feat_mean[:,1], new_feat_mean[:,2], label=file_list_csv[idx][:-4], c = color[idx])
plt.title('mag comparison')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …